In [1361]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from string import punctuation
from itertools import chain
from string import digits

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, accuracy_score,confusion_matrix, classification_report

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_selection import SelectPercentile, chi2, f_regression, f_classif

from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TweetTokenizer
from emoji import demojize

import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [1301]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin1') 

In [1302]:
df1 = pd.read_csv('testdata.manual.2009.06.14.csv') 

In [1303]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [1304]:
df.columns = ['polarity', 'tweet_id', 'date', 'query', 'user', 'tweet',]

In [1305]:
df1.columns = ['polarity', 'tweet_id', 'date', 'query', 'user', 'tweet',]

In [1306]:
df1.head()

,polarity,tweet_id,date,query,user,tweet
0,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
1,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
3,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
4,4,8,Mon May 11 03:22:00 UTC 2009,kindle2,GeorgeVHulme,@richardebaker no. it is too big. I'm quite ha...


In [1307]:
df.head()

,polarity,tweet_id,date,query,user,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


***
### 1. Create sentiment column to classify the sentiment of tweets using the polarity

According to the dataset, 0 means negative, 2 means neutral and 4 means positive. We do not need neutral tweets as they do not add value to our analysis of the tweet sentiment, so it would've been dropped, however, the dataset already had it dropped.

In [1308]:
pos = df[df['polarity'] == 4] 
neg = df[df['polarity'] == 0]
neutral = df[df['polarity']==2]
print(pos.shape, neg.shape, neutral.shape)
#Gives us insight into the ratings and how they shape the data. 

(800000, 6) (799999, 6) (0, 6)


In [1309]:
df = shuffle(df) #shuffled to make the data more represntative when used for the following operations of testing, training etc.
df.head()
#Reference: https://towardsdatascience.com/shuffling-rows-in-pandas-dataframes-eda052275635

,polarity,tweet_id,date,query,user,tweet
1369784,4,2050886412,Fri Jun 05 19:28:18 PDT 2009,NO_QUERY,insearchofnkotb,Joe and jon squeezing donnie's ass in the hudd...
1010430,4,1881000834,Fri May 22 03:15:14 PDT 2009,NO_QUERY,vastvision,Peter Martijn Wijnia - Sacrificies (Vast Visio...
285474,0,1993620860,Mon Jun 01 10:40:14 PDT 2009,NO_QUERY,keriwgd,So anxious about the Condo. I'm so hoping we d...
423128,0,2062807404,Sun Jun 07 00:21:56 PDT 2009,NO_QUERY,kissypoo416,Pissed my bf stole my toothpaste GRRR.... It h...
1184770,4,1982665932,Sun May 31 11:39:45 PDT 2009,NO_QUERY,MouseGoesSqueak,"@Anticlimatic Hehe. Speaking of eclipses, your..."


In [1310]:
y_map = {0:0, 4:1} #Map out values, 1 and 2 should map to 0 meaning a negative review, and 4 and 5 map to 1 for pos.
y = df['polarity'].map(y_map) #Now if we do df['column'].map we transform the data 1,2,4,5 into 0 and 1 making it binary

In [1311]:
df['sentiment'] = y

In [1312]:
df.head()

,polarity,tweet_id,date,query,user,tweet,sentiment
1369784,4,2050886412,Fri Jun 05 19:28:18 PDT 2009,NO_QUERY,insearchofnkotb,Joe and jon squeezing donnie's ass in the hudd...,1
1010430,4,1881000834,Fri May 22 03:15:14 PDT 2009,NO_QUERY,vastvision,Peter Martijn Wijnia - Sacrificies (Vast Visio...,1
285474,0,1993620860,Mon Jun 01 10:40:14 PDT 2009,NO_QUERY,keriwgd,So anxious about the Condo. I'm so hoping we d...,0
423128,0,2062807404,Sun Jun 07 00:21:56 PDT 2009,NO_QUERY,kissypoo416,Pissed my bf stole my toothpaste GRRR.... It h...,0
1184770,4,1982665932,Sun May 31 11:39:45 PDT 2009,NO_QUERY,MouseGoesSqueak,"@Anticlimatic Hehe. Speaking of eclipses, your...",1


In [1313]:
df.dtypes

polarity      int64
tweet_id      int64
date         object
query        object
user         object
tweet        object
sentiment     int64
dtype: object

***
### 2. Cleaning the data

1. Convert to lowercase
2. Rmove numbers
3. Remove punctuation marks 
4. Remove HTML tags
5. Convert emoticons to strings
6. Tokenize the data
7. Remove stop words
8. Stemming

In [1314]:
def remove_usernames_links(text):
    text = re.sub('@[^\s]+','',str(text))
    text = re.sub('http[^\s]+','',str(text))
    return text

In [1315]:
def convert_to_lowercase(text):
    return text.lower()

In [1316]:
def remove_nums(text):
    no_nums = "".join([i for i in text if i not in string.digits])
    return no_nums

In [1317]:
def remove_punctuations(text):
    no_puncts = "".join([c for c in text if c not in string.punctuation])
    return no_puncts

In [1318]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'lxml')
    no_html_tags = soup.get_text()
    return no_html_tags

In [1320]:
def remove_emojis(text):
    no_emoji = demojize(text)
    return no_emoji

In [1321]:
tokenizer = RegexpTokenizer('\s+', gaps = True)

In [1322]:
tk = TweetTokenizer()

In [1323]:
en_stopwords = set(stopwords.words('english'))

In [1324]:
def remove_stopwords(text):
    words = [w for w in text if w not in en_stopwords]
    return words

In [1325]:
lemmatizer = WordNetLemmatizer()

In [1326]:
def word_lemmatizer(text):
    lemmatized_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lemmatized_text

In [1327]:
def clean_text(text):
    cleaned_text = text.apply(remove_usernames_links)
    cleaned_text = cleaned_text.apply(convert_to_lowercase)
    cleaned_text = cleaned_text.apply(remove_nums)
    cleaned_text = cleaned_text.apply(remove_emojis) 
    cleaned_text = cleaned_text.apply(remove_punctuations)
    cleaned_text = cleaned_text.apply(remove_html_tags)
    cleaned_text = cleaned_text.apply(tokenizer.tokenize)
    cleaned_text = cleaned_text.apply(lambda x: remove_stopwords(x))
    cleaned_text = cleaned_text.apply(lambda x: word_lemmatizer(x))
    return cleaned_text

In [1328]:
def clean(text):
    cleaned_text = []
    
    #text = text.lower()
    text = remove_emojis(text)
    text = remove_nums(text)
    text = remove_punctuations(text)
    text = remove_html_tags(text)
    text = tokenizer.tokenize(text)
    text = remove_stopwords(text)
    text = word_lemmatizer(text)
    
    cleaned_text.append(text)
    
    return cleaned_text

In [1329]:
dftext = df['tweet']
dftext

1369784    Joe and jon squeezing donnie's ass in the hudd...
1010430    Peter Martijn Wijnia - Sacrificies (Vast Visio...
285474     So anxious about the Condo. I'm so hoping we d...
423128     Pissed my bf stole my toothpaste GRRR.... It h...
1184770    @Anticlimatic Hehe. Speaking of eclipses, your...
                                 ...                        
20382                              Still can't fall asleep. 
272275     @Tarale That's really bad, very sorry 4 you.  ...
1479373    happy that Boom Boom Pow is number 1 in the UK...
143503                                Is tired from traffic 
501401     I miss my younger days when &quot;the last day...
Name: tweet, Length: 1599999, dtype: object

In [1330]:
cleaned_tweets = clean_text(dftext)

In [1331]:
df['cleaned_tweet'] = cleaned_tweets

In [1333]:
df.head()

,polarity,tweet_id,date,query,user,tweet,sentiment,cleaned_tweet
1369784,4,2050886412,Fri Jun 05 19:28:18 PDT 2009,NO_QUERY,insearchofnkotb,Joe and jon squeezing donnie's ass in the hudd...,1,joe jon squeezing donnies as huddle bout nd pose
1010430,4,1881000834,Fri May 22 03:15:14 PDT 2009,NO_QUERY,vastvision,Peter Martijn Wijnia - Sacrificies (Vast Visio...,1,peter martijn wijnia sacrificies vast vision r...
285474,0,1993620860,Mon Jun 01 10:40:14 PDT 2009,NO_QUERY,keriwgd,So anxious about the Condo. I'm so hoping we d...,0,anxious condo im hoping dont lose
423128,0,2062807404,Sun Jun 07 00:21:56 PDT 2009,NO_QUERY,kissypoo416,Pissed my bf stole my toothpaste GRRR.... It h...,0,pissed bf stole toothpaste grrr oxidating bubb...
1184770,4,1982665932,Sun May 31 11:39:45 PDT 2009,NO_QUERY,MouseGoesSqueak,"@Anticlimatic Hehe. Speaking of eclipses, your...",1,hehe speaking eclipse vagina


In [1334]:
###TEST CLASSIFICATION

***
### Classification

In [1335]:
#Reference: Lab 5
def text_fit(X, y, model, clf_model, coef_show=1): 
    X_c = model.fit_transform(X) 
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0) #Splits the data into one for training and one for testing
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    clf = clf_model.fit(X_train, y_train) #Fit the logistic reg model with the training data
    y_pred = clf.predict(X_test)  #Prediction using the Test data
    if coef_show == 1:  #Extract the coefficients from the model and put it in a dataframe
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('-Top 20 positive-')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('-Top 20 negative-')        
        print(coeff_df.tail(20).to_string(index=False))

In [1336]:
X = df['cleaned_tweet']
y = df['sentiment']

In [1337]:
tfidf_n = TfidfVectorizer(ngram_range=(1,2))

In [1338]:
svm_tfidf_n = text_fit(X, y, tfidf_n, LinearSVC())

# features: 4397916
# train records: 1199999
# test records: 400000


C:\Users\adria\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



-Top 20 positive-
             Word  Coefficient
        cant wait     5.445716
      cannot wait     4.234849
      doesnt hurt     3.437968
wont disappointed     3.297625
        wish luck     3.218377
         aint bad     3.165187
    nothing wrong     3.131649
        wont hurt     3.052176
       get enough     3.047692
      sad anymore     2.910309
          smiling     2.865787
            smile     2.791474
         cant bad     2.787228
    headache gone     2.716984
        canâ wait     2.706236
         isnt bad     2.680736
        wasnt bad     2.633715
      wont regret     2.625691
      take credit     2.609796
          sad sad     2.595777

-Top 20 negative-
         Word  Coefficient
       broken    -3.494878
       missin    -3.563985
disappointing    -3.650849
  passed away    -3.658302
     headache    -3.658768
unfortunately    -3.727078
        upset    -3.811648
         cant    -3.872348
         lost    -3.886955
       ruined    -3.931638
       bummer 

In [1339]:
lr_tfidf_n = text_fit(X, y, tfidf_n, LogisticRegression())

# features: 4397916
# train records: 1199999
# test records: 400000


C:\Users\adria\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\adria\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)



-Top 20 positive-
          Word  Coefficient
     cant wait    13.678785
   cannot wait    11.512841
     wish luck     9.485170
       smiling     8.558742
     wasnt bad     8.157187
congratulation     8.011944
 nothing wrong     7.822763
      isnt bad     7.734759
          glad     7.164461
     dont miss     6.990786
    get enough     6.960296
         smile     6.955471
       welcome     6.930817
          love     6.918243
       blessed     6.877666
       excited     6.844365
         thank     6.795414
       amazing     6.784287
   dont forget     6.762348
         proud     6.564884

-Top 20 negative-
         Word  Coefficient
unfortunately   -10.431982
         hurt   -10.443813
        upset   -10.626204
       bummer   -10.693473
         lost   -10.701497
       ruined   -10.715436
    depressed   -10.721984
   depressing   -10.746080
         miss   -10.805262
 disappointed   -10.845649
         poor   -11.110749
    cancelled   -11.329789
       gutted   -11.489

***
### Topic Modelling Using NMF

In [1340]:
#Reference: Lab 5
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print("\n")
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])
            print("\n")

In [1341]:
documents = list(X)
print(len(documents))

1599999


In [1342]:
#TFIDF model using NMF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

C:\Users\adria\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [1343]:
no_topics = 10

In [1344]:
#Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

C:\Users\adria\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
C:\Users\adria\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
C:\Users\adria\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [1345]:
no_top_words = 20
no_top_documents = 8
print("NMF Topics \n\n")
display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)

NMF Topics 


Topic 0:
day today happy great mother last school another beautiful long nice birthday tomorrow bad everyone hope first sunny mom father


day


day


day


day


day


day


day


selfdeclared day


Topic 1:
good morning night luck everyone thats hope sound feel feeling thing time today world pretty afternoon monday weekend idea say


good night good morning


good morning good night


good night good morning


good morning good night


good morning good night


good night good morning


good morning good night


good morning good night


Topic 2:
thanks follow following followfriday much great ff hey haha guy ill lol awesome aww lot ok link appreciate cool sharing


royhooker thanks


thanks


thanks


thanks


thanks


thanks


thanks quotfucktasticquot


thanks


Topic 3:
im sorry going tired gonna sad bored sick sure still getting think happy right excited glad bed hungry hear home


im bitcham


im tnulb


im


im


im


im


im


im


Topic 4:
get cant like time lo

***
### Classifying whether a user's tweet has a negative or positive sentiment

In [1346]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)

In [1347]:
tfidf_n.fit(X_train)

TfidfVectorizer(ngram_range=(1, 2))

In [1348]:
X_train = tfidf_n.transform(X_train)
X_test  = tfidf_n.transform(X_test)

In [1349]:
def assess(clf_model):
    
    y_pred = clf_model.predict(X_test)
    recall = recall_score(y_test,y_pred) #Calculate the recall score between actual predictions and model predictions
    accuracy = accuracy_score(y_test,y_pred)
    
    print(classification_report(y_test, y_pred))
    print ('Model Recall: {}'.format(recall))
    print ('Model Accuracy: {}'.format(accuracy))

In [1350]:
svc_model = LinearSVC().fit(X_train, y_train)
assess(svc_model)

              precision    recall  f1-score   support

           0       0.79      0.79      0.79     39880
           1       0.79      0.79      0.79     40120

    accuracy                           0.79     80000
   macro avg       0.79      0.79      0.79     80000
weighted avg       0.79      0.79      0.79     80000

Model Recall: 0.7944915254237288
Model Accuracy: 0.7925


In [1351]:
lgr_model = LogisticRegression().fit(X_train, y_train)
assess(lgr_model)

              precision    recall  f1-score   support

           0       0.80      0.80      0.80     39880
           1       0.80      0.80      0.80     40120

    accuracy                           0.80     80000
   macro avg       0.80      0.80      0.80     80000
weighted avg       0.80      0.80      0.80     80000

Model Recall: 0.803988035892323
Model Accuracy: 0.8003


C:\Users\adria\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [1352]:
def predict_sentiment(tfidf_n, model, text):
    tweet = tfidf_n.transform(clean(text))
    sentiment = model.predict(tweet)
    posp = "Positive Sentiment"
    negp = "Negative Sentiment"
    
    if sentiment == 0:
        return negp
    if sentiment == 1:
        return posp

In [1357]:
#Enter your tweet to predict the sentiment:
tweet = ["I hate puppies"]
predict_sentiment(tfidf_n, svc_model, tweet)

'Negative Sentiment'

In [1358]:
#Enter your tweet to predict the sentiment:
tweet = ["I love puppies"]
predict_sentiment(tfidf_n, svc_model, tweet)

'Positive Sentiment'

In [1355]:
#Enter your tweet to predict the sentiment:
tweet = ["I hate puppies"]
predict_sentiment(tfidf_n, lgr_model, tweet)

'Negative Sentiment'

In [1359]:
#Enter your tweet to predict the sentiment:
tweet = ["I love puppies"]
predict_sentiment(tfidf_n, lgr_model, tweet)

'Positive Sentiment'

References:

* Lab 5
* https://dylancastillo.co/nlp-snippets-clean-and-tokenize-text-with-python/#transform-emojis-to-characters
* https://machinelearningmastery.com/how-to-connect-model-input-data-with-predictions-for-machine-learning/
* https://www.academia.edu/74585014/Machine_Learning_Approach_to_Sentiment_Analysis_from_Movie_Reviews_Using_Word2Vec
* https://entertainment.bacsigan.com/popedaze/sentiment-analysis-in-python
* https://www.w3schools.com/python/python_lists_add.asp
* https://stackoverflow.com/questions/32106063/sklearn-linearsvc-x-has-1-features-per-sample-expecting-5
* https://datascience.stackexchange.com/questions/51224/why-does-transform-from-tfidf-vectorizer-sklearn-not-work